In [1]:
import random
import math
from IPython.display import display
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

In [2]:
class TicTacToe_Game:
    
    def initialise_baord(self) : 
        self.ttt_board = {
                            1: ' ', 2:' ', 3: ' ',
                            4: ' ', 5:' ', 6: ' ',
                            7: ' ', 8:' ', 9: ' '
                         }
    def display_board(self):
        print("\n")
        for row in range(3):
            for col in range(3):
                cell = row * 3 + col + 1
                print(self.ttt_board[cell], end="")
                if col < 2:
                    print(" | ", end="")
            print()
            if row < 2:
                print("---------")
        print()


    def tossForFirstMove(self):
        choices = [1,2]
        return random.choice(choices)
    
    def get_random_generated_move(self):
        position = random.randint(1, 9)
        if self.validateMove(position):
            return position
        else:
            position = self.get_random_generated_move()
            return position
        
    def display_board(self):
        print("\n")
        print( self.ttt_board[1], '|', self.ttt_board[2], '|', self.ttt_board[3])
        print(' -+---+-')
        print(self.ttt_board[4], '|', self.ttt_board[5], '|', self.ttt_board[6])
        print(' -+---+-')
        print(self.ttt_board[7], '|', self.ttt_board[8], '|', self.ttt_board[9], "\n")
        
    def validateMove(self, move):
        return self.ttt_board[move] == ' '
        
    def validateDraw(self):
        return all(self.ttt_board[key] != ' ' for key in self.ttt_board.keys())
    
    
    def validateWin(self):
        win_combinations = [
            (1, 2, 3), (4, 5, 6), (7, 8, 9), 
            (1, 4, 7), (2, 5, 8), (3, 6, 9),  
            (1, 5, 9), (7, 5, 3)
        ]

        for combo in win_combinations:
            if (self.ttt_board[combo[0]] == self.ttt_board[combo[1]] == self.ttt_board[combo[2]] != ' '):
                return True

        return False
                        
    def validateWinForLetter(self, mark):
        winning_positions = [
            (1, 2, 3), (4, 5, 6), (7, 8, 9),
            (1, 4, 7), (2, 5, 8), (3, 6, 9),
            (1, 5, 9), (7, 5, 3)
        ]
        for pos in winning_positions:
            if all(self.ttt_board[i] == mark for i in pos):
                return True
        return False

    


In [3]:
class QLearning:
    def __init__(self):
        self.epsilon = 1.0
        self.QLearningStates = {}
  
    getPosition = lambda self, current_board: tuple(tuple(current_board[i+j] for j in range(3)) for i in range(1, 10, 3))

    def getQLearningValue_For_Action(self, current_board, current_position):
        position = self.getPosition(current_board)
        if position not in self.QLearningStates:
            self.QLearningStates[position] = np.zeros((9,))
        return self.QLearningStates[position][current_position - 1]
  

    def getBestPositionFromQLearning(self, current_board, possible_positions):
        return random.choice(possible_positions) if random.random() < self.epsilon else max(possible_positions, key=lambda x: self.getQLearningValue_For_Action(current_board, x))
            
    def loadQLearningModel(self):
        with open("TicTacToeQLearningModel.pickle", "rb") as file:
            self.QLearningStates = pickle.load(file)
                    
    

In [4]:
class TicTacToe_MinMax:
    
    def Min_Max_Move_with_alpha_beta_pruning(self, ttt_game, MinMax_Letter, QLearing_Letter):
        optimised_score =  -math.inf
        optimised_position = ttt_game.get_random_generated_move()

        for possible_position in ttt_game.ttt_board.keys() : 

            if ttt_game.ttt_board[possible_position] == ' ' :
                ttt_game.ttt_board[possible_position] = MinMax_Letter
                current_score = self.evaluate_MinMax_score_with_alpha_beta_pruning(ttt_game, MinMax_Letter, QLearing_Letter, False, -math.inf, math.inf)
                ttt_game.ttt_board[possible_position] = ' '

                if current_score > optimised_score :
                    optimised_score = current_score
                    optimised_position = possible_position
        
        return optimised_position
    
    
    def evaluate_MinMax_score_with_alpha_beta_pruning(self, ttt_game, MinMax_Letter, QLearing_Letter, isMinMaxMove, alpha, beta):
        if ttt_game.validateWinForLetter(MinMax_Letter) :
            return 1
        elif ttt_game.validateWinForLetter(QLearing_Letter) :
            return -1
        elif ttt_game.validateDraw() :
            return 0

        if isMinMaxMove :
            optimisedScore = -math.inf

            for possible_position in ttt_game.ttt_board.keys():

                if ttt_game.ttt_board[possible_position] == ' ' :
                    ttt_game.ttt_board[possible_position] = MinMax_Letter
                    current_score = self.evaluate_MinMax_score_with_alpha_beta_pruning(ttt_game, MinMax_Letter, QLearing_Letter, False, alpha, beta)
                    ttt_game.ttt_board[possible_position] = ' '

                    optimisedScore = max(optimisedScore, current_score)
                    alpha = max(alpha, optimisedScore)

                    if alpha >= beta :
                        break

            return optimisedScore

        else:
            optimisedScore = math.inf
            
            for possible_position in ttt_game.ttt_board.keys():
                if ttt_game.ttt_board[possible_position] == ' ':
                    ttt_game.ttt_board[possible_position] = QLearing_Letter
                    current_score = self.evaluate_MinMax_score_with_alpha_beta_pruning(ttt_game, MinMax_Letter, QLearing_Letter, True, alpha, beta)
                    ttt_game.ttt_board[possible_position] = ' '

                    optimisedScore = min(optimisedScore, current_score)
                    beta = min(beta, optimisedScore)

                    if alpha >= beta :
                        break

            return optimisedScore

In [5]:

def play_tic_tac_toe(MinMaxPlaysFirst, QLearning, MinMax, ttt_game):
        MinMaxLetter = 'O'
        QLearning_Letter = 'X'

        while True:
            if MinMaxPlaysFirst:
                
                MinMaxPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(MinMaxPossible_Positions) == 0:
                    return "Draw"
            
                MinMaxPosition = MinMax.Min_Max_Move_with_alpha_beta_pruning(ttt_game, MinMaxLetter, QLearning_Letter)
                
                if ttt_game.validateMove(MinMaxPosition):
                    ttt_game.ttt_board[MinMaxPosition] = MinMaxLetter
      
                if ttt_game.validateWinForLetter(MinMaxLetter) : 
                    return "MinMaxWon"

                if ttt_game.validateDraw():
                    return "Draw"

                QLearningPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(QLearningPossible_Positions) == 0:
                    break

                QLearningPosition = QLearning.getBestPositionFromQLearning(ttt_game.ttt_board, QLearningPossible_Positions)

                if ttt_game.validateMove(QLearningPosition):
                    ttt_game.ttt_board[QLearningPosition] = QLearning_Letter

                if ttt_game.validateWinForLetter(QLearning_Letter) : 
                    return "QLearningWon"

                if ttt_game.validateDraw():
                    return "Draw"

            else:
                QLearningPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(QLearningPossible_Positions) == 0:
                    break

                QLearningPosition = QLearning.getBestPositionFromQLearning(ttt_game.ttt_board, QLearningPossible_Positions)

                if ttt_game.validateMove(QLearningPosition):
                    ttt_game.ttt_board[QLearningPosition] = QLearning_Letter

                if ttt_game.validateWinForLetter(QLearning_Letter) : 
                    return "QLearningWon"

                if ttt_game.validateDraw():
                    return "Draw"


                MinMaxPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(MinMaxPossible_Positions) == 0:
                    return "Draw"
            
                MinMaxPosition = MinMax.Min_Max_Move_with_alpha_beta_pruning(ttt_game, MinMaxLetter, QLearning_Letter)
                
                if ttt_game.validateMove(MinMaxPosition):
                    ttt_game.ttt_board[MinMaxPosition] = MinMaxLetter
      
                if ttt_game.validateWinForLetter(MinMaxLetter) : 
                    return "MinMaxWon"

                if ttt_game.validateDraw():
                    return "Draw"
                

### First Move: Random

In [6]:
games = 2000
MinMaxWin = QLearningWin = Draw = 0

qLearningPlayer = QLearning() 
qLearningPlayer.loadQLearningModel()

ttt_min_max = TicTacToe_MinMax()
print(f"Current Q Learning model has {len(qLearningPlayer.QLearningStates)} states")

for _ in tqdm(range(games)):
    ttt_game = TicTacToe_Game()
    ttt_game.initialise_baord()
    
    MinMaxPlaysFirst = False
    if ttt_game.tossForFirstMove() == 1 :
        MinMaxPlaysFirst = True
    else : 
        MinMaxPlaysFirst = False
    
    
    winner = play_tic_tac_toe(MinMaxPlaysFirst, qLearningPlayer, ttt_min_max, ttt_game)
    
    if winner == 'QLearningWon':
        QLearningWin += 1
    elif winner == 'MinMaxWon':
        MinMaxWin += 1
    else:
        Draw += 1



  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

Current Q Learning model has 5020 states


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [07:56<00:00,  4.19it/s]


In [9]:
statistics_df = pd.DataFrame(columns=['Game Type', 'Total Number of Games', 'Number of Games QLearning Won', 'Number of Games MinMax Won', 'Number of Games Drawn'])
statistics_dict = {}
statistics_dict['Game Type'] = 'First Move: Random '
statistics_dict['Total Number of Games'] = games
statistics_dict['Number of Games QLearning Won'] = QLearningWin
statistics_dict['Number of Games MinMax Won'] = MinMaxWin
statistics_dict['Number of Games Drawn'] = Draw
statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
statistics_df = statistics_df.style.applymap(lambda x:'white-space:nowrap')
display(statistics_df)

,Game Type,Total Number of Games,Number of Games QLearning Won,Number of Games MinMax Won,Number of Games Drawn
0,First Move: Random,2000,206,1196,598


### First Move: Always Q-Learning Player

In [10]:
games = 2000
MinMaxWin = QLearningWin = Draw = 0

qLearningPlayer = QLearning() 
qLearningPlayer.loadQLearningModel()

ttt_min_max = TicTacToe_MinMax()
print(f"Current Q Learning model has {len(qLearningPlayer.QLearningStates)} states")

for _ in tqdm(range(games)):
    ttt_game = TicTacToe_Game()
    ttt_game.initialise_baord()
    
    MinMaxPlaysFirst = False
    
    winner = play_tic_tac_toe(MinMaxPlaysFirst, qLearningPlayer, ttt_min_max, ttt_game)
    
    if winner == 'QLearningWon':
        QLearningWin += 1
    elif winner == 'MinMaxWon':
        MinMaxWin += 1
    else:
        Draw += 1



  0%|                                                                                 | 1/2000 [00:00<03:57,  8.40it/s]

Current Q Learning model has 5020 states


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [03:00<00:00, 11.09it/s]


In [12]:
statistics_df = pd.DataFrame(columns=['Game Type', 'Total Number of Games', 'Number of Games QLearning Won', 'Number of Games MinMax Won', 'Number of Games Drawn'])
statistics_dict = {}
statistics_dict['Game Type'] = 'First Move: Q-Learning Player'
statistics_dict['Total Number of Games'] = games
statistics_dict['Number of Games QLearning Won'] = QLearningWin
statistics_dict['Number of Games MinMax Won'] = MinMaxWin
statistics_dict['Number of Games Drawn'] = Draw
statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
statistics_df = statistics_df.style.applymap(lambda x:'white-space:nowrap')
display(statistics_df)

,Game Type,Total Number of Games,Number of Games QLearning Won,Number of Games MinMax Won,Number of Games Drawn
0,First Move: Q-Learning Player,2000,318,906,776


### First Move: Always Min-Max Player

In [13]:
games = 2000
MinMaxWin = QLearningWin = Draw = 0

qLearningPlayer = QLearning() 
qLearningPlayer.loadQLearningModel()

ttt_min_max = TicTacToe_MinMax()
print(f"Current Q Learning model has {len(qLearningPlayer.QLearningStates)} states")

for _ in tqdm(range(games)):
    ttt_game = TicTacToe_Game()
    ttt_game.initialise_baord()
    
    MinMaxPlaysFirst = True
    
    winner = play_tic_tac_toe(MinMaxPlaysFirst, qLearningPlayer, ttt_min_max, ttt_game)

    if winner == 'QLearningWon':
        QLearningWin += 1
    elif winner == 'MinMaxWon':
        MinMaxWin += 1
    else:
        Draw += 1



  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

Current Q Learning model has 5020 states


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [12:15<00:00,  2.72it/s]


In [16]:
statistics_df = pd.DataFrame(columns=['Game Type', 'Total Number of Games', 'Number of Games QLearning Won', 'Number of Games MinMax Won', 'Number of Games Drawn'])
statistics_dict = {}
statistics_dict['Game Type'] = 'First Move: Min-Max Player'
statistics_dict['Total Number of Games'] = games
statistics_dict['Number of Games QLearning Won'] = QLearningWin
statistics_dict['Number of Games MinMax Won'] = MinMaxWin
statistics_dict['Number of Games Drawn'] = Draw
statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
statistics_df = statistics_df.style.applymap(lambda x:'white-space:nowrap')
display(statistics_df)

,Game Type,Total Number of Games,Number of Games QLearning Won,Number of Games MinMax Won,Number of Games Drawn
0,First Move: Min-Max Player,2000,0,1809,191
